# pyAutoTradingBot

## Open Trade Strategy - Import Data

In [ ]:
import os
import pandas as pd
from PIL import Image, ImageDraw

In [ ]:
MAX_GAIN = 0.01
MAX_LOSS = 0.005

color_for_trade = {}
color_for_trade['S'] = (255,0,0)
color_for_trade['L'] = (0,255,0)
color_for_trade['H'] = (255,255,0)

In [ ]:

csv_diario = pd.read_csv(os.path.join('input','WDOFUT_F_0_Diario.train.csv'), sep=';')
csv_15min = pd.read_csv(os.path.join('input','WDOFUT_F_0_15min.train.csv'), sep=';') 

data_daily = csv_diario.to_dict('records')
data_15min = csv_15min.to_dict('records')

data_daily = data_daily[1:]
data_15min = data_15min[1:]

In [ ]:
def convert(x):
    return float(x.replace(',','.'))

def convert_date(x):
    tokens = x.split('/')
    return tokens[2] + tokens[1] + tokens[0]

def convert_time(x):
    tokens = x.split(":")
    return int(tokens[0] + tokens[1])

In [ ]:
data_15min = [{'Asset': d['Asset'],
               'Date': convert_date(d['Date']), 
               'Time': convert_time(d['Time']), 
               'Open': convert(d['Open']), 
               'High': convert(d['High']), 
               'Low': convert(d['Low']), 
               'Close': convert(d['Close']), 
               'Volume': convert(d['Volume']),
               'Trades': d['Trades']}
               for d in data_15min]
data_daily = [{'Asset': d['Asset'],
               'Date': convert_date(d['Date']), 
               'Open': convert(d['Open']), 
               'High': convert(d['High']), 
               'Low': convert(d['Low']), 
               'Close': convert(d['Close']), 
               'Volume': convert(d['Volume']),
               'Trades': d['Trades']}
               for d in data_daily]

sorted(data_15min, key=lambda x: (x['Date'], x['Time']), reverse=True)
sorted(data_daily, key=lambda x: x['Date'], reverse=True)

days_in_15min = sorted(set([x['Date'] for x in data_15min]))
days_in_daily = sorted(set([x['Date'] for x in data_daily]))

data_daily = [x for x in data_daily if x['Date'] in days_in_15min]

In [ ]:
def clear_output():
    for parent, dirnames, filenames in os.walk('output'):
        for filename in filenames:
            if filename.lower().endswith('.png'):
                file_name=os.path.join(parent, filename)
                #print('remove ' + file_name)

In [ ]:
def get_correct_trade(info):
    correct_candles = [x for x in data_15min if x['Date'] == info['date']]
    correct_candles.sort(key=lambda x: x['Time'])
    current_open = correct_candles[0]['Open']
    current_target_long = current_open * (1 + MAX_GAIN)
    current_target_short = current_open * (1 - MAX_GAIN)
    check_if_long = [x for x in correct_candles if x['High'] > current_target_long and x['Time'] < 1400]
    check_if_short = [x for x in correct_candles if x['Low'] < current_target_short and x['Time'] < 1400]
    if check_if_long and check_if_short:
       if int(check_if_long[0]['Time']) < int(check_if_short[0]['Time']):
          return 'L', check_if_long, check_if_short, current_target_long, current_target_short
       else:
          return 'S', check_if_long, check_if_short, current_target_long, current_target_short
    if check_if_long:
       return 'L', check_if_long, check_if_short, current_target_long, current_target_short
    if check_if_short:
       return 'S', check_if_long, check_if_short, current_target_long, current_target_short
    return 'H', [], [], current_target_long, current_target_short


In [ ]:
returned_coefs = []

for i, current_day in enumerate(data_daily):
    if i > 3:
        first_open = data_daily[i-3]['Open']

        current_date = data_daily[i]['Date']

        open_x1 = 1
        high_x1 = ((data_daily[i-3]['High'] / first_open) - 1) * 100 
        close_x1 = ((data_daily[i-3]['Close'] /first_open) - 1) * 100
        low_x1 = ((data_daily[i-3]['Low'] / first_open) - 1) * 100

        last_close = data_daily[i-3]['Close']

        open_x2 = ((data_daily[i-2]['Open'] / last_close) -1) * 100
        high_x2 = ((data_daily[i-2]['High'] / last_close) -1) * 100
        close_x2 = ((data_daily[i-2]['Close'] / last_close) -1) * 100
        low_x2 = ((data_daily[i-2]['Low'] / last_close) -1) * 100

        last_close = data_daily[i-2]['Close']

        open_x3 = ((data_daily[i-1]['Open'] / last_close) -1) * 100
        high_x3 = ((data_daily[i-1]['High'] / last_close) -1) * 100
        close_x3 = ((data_daily[i-1]['Close'] / last_close) -1) * 100
        low_x3 = ((data_daily[i-1]['Low'] / last_close) -1) * 100

        last_close = data_daily[i-1]['Close']

        current_open = ((data_daily[i]['Open'] / last_close) - 1) * 100

        current_values = [open_x1, high_x1, close_x1, low_x1, open_x2, high_x2, close_x2,
                          low_x2, open_x3, high_x3, close_x3, low_x3, current_open]

        current_values_abs=[abs(x) for x in current_values]

        new_coef = {
            'date': current_date,
            'first_open': first_open,

            'open_x1': open_x1,
            'high_x1': high_x1,
            'close_x1': close_x1,
            'low_x1': low_x1,

            'open_x2': open_x2,
            'high_x2': high_x2,
            'close_x2': close_x2,
            'low_x2': low_x2,

            'open_x3': open_x3,
            'high_x3': high_x3,
            'close_x3': close_x3,
            'low_x3': low_x3,

            'current_open' : current_open,
            'values': current_values,
            'absvalues': current_values_abs
            }
        
        trade, long, short, target_long, target_short = get_correct_trade(new_coef)

        new_coef['trade'] =trade
        new_coef['long'] = long
        new_coef['short'] = short
        new_coef['target_long'] = target_long
        new_coef['target_short'] = target_short

        returned_coefs.append(new_coef)

In [ ]:
clear_output()

for current_x in returned_coefs:
    img = Image.new("RGB", (210,200), (255,255,255))
    img1 = ImageDraw.Draw(img)
    max_y = max(current_x['absvalues'])

    current_open = 100
    current_close = current_x['close_x1']
    negative = current_close > 0
    current_y = abs(current_close) / max_y
    current_y_pos = 100 - (current_y * 100) if negative else 100 + (current_y * 100)
    img1.line(((5 + (55*0),100), (50 + (55*0), current_y_pos )), (255,0,0), 2)
    img1.ellipse(((45 + (55*0),current_y_pos - 5),(55 + (55*0),current_y_pos + 5)), (255,0,0), 2)

    current_high = current_x['high_x1']
    current_low = current_x['low_x1']
    negative_high = current_high > 0
    negative_low = current_low > 0
    current_y_high = abs(current_high) / max_y
    current_y_low = abs(current_low) / max_y
    current_y_high_pos = 100 - (current_y_high * 100) if negative_high else 100 + (current_y_high * 100)
    current_y_low_pos = 100 - (current_y_low * 100) if negative_low else 100 + (current_y_low * 100)
    img1.line(((20 + (55*0),current_y_low_pos), (40 + (55*0), current_y_high_pos)), (255,0,0), 2)

    current_open = current_x['open_x2']
    current_close = current_x['close_x2']
    negative = current_close > 0
    current_y = abs(current_close) / max_y
    current_y_pos = 100 - (current_y * 100) if negative else 100 + (current_y * 100)
    current_y_open = abs(current_open) / max_y
    current_y_pos_open = 100 - (current_y_open * 100) if negative else 100 + (current_y_open * 100)
    img1.line(((5 + (55*1),current_y_pos_open), (50 + (55*1), current_y_pos )), (255,0,0), 2)
    img1.ellipse(((45 + (55*1),current_y_pos - 5),(55 + (55*1),current_y_pos + 5)), (255,0,0), 2)

    current_high = current_x['high_x2']
    current_low = current_x['low_x2']
    negative_high = current_high > 0
    negative_low = current_low > 0
    current_y_high = abs(current_high) / max_y
    current_y_low = abs(current_low) / max_y
    current_y_high_pos = 100 - (current_y_high * 100) if negative_high else 100 + (current_y_high * 100)
    current_y_low_pos = 100 - (current_y_low * 100) if negative_low else 100 + (current_y_low * 100)
    img1.line(((20 + (55*1),current_y_low_pos), (40 + (55*1), current_y_high_pos)), (255,0,0), 2)

    current_open = current_x['open_x3']
    current_close = current_x['close_x3']
    negative = current_close > 0
    current_y = abs(current_close) / max_y
    current_y_pos = 100 - (current_y * 100) if negative else 100 + (current_y * 100)
    current_y_open = abs(current_open) / max_y
    current_y_pos_open = 100 - (current_y_open * 100) if negative else 100 + (current_y_open * 100)
    img1.line(((5 + (55*2),current_y_pos_open), (50 + (55*2), current_y_pos )), (255,0,0), 2)
    img1.ellipse(((45 + (55*2),current_y_pos - 5),(55 + (55*2),current_y_pos + 5)), (255,0,0), 2)

    current_high = current_x['high_x3']
    current_low = current_x['low_x3']
    negative_high = current_high > 0
    negative_low = current_low > 0
    current_y_high = abs(current_high) / max_y
    current_y_low = abs(current_low) / max_y
    current_y_high_pos = 100 - (current_y_high * 100) if negative_high else 100 + (current_y_high * 100)
    current_y_low_pos = 100 - (current_y_low * 100) if negative_low else 100 + (current_y_low * 100)
    img1.line(((20 + (55*2),current_y_low_pos), (40 + (55*2), current_y_high_pos)), (255,0,0), 2)

    current_open = current_x['current_open']
    current_close = current_x['current_open']
    negative = current_close > 0
    current_y = abs(current_close) / max_y
    current_y_pos = 100 - (current_y * 100) if negative else 100 + (current_y * 100)
    current_y_open = abs(current_open) / max_y
    current_y_pos_open = 100 - (current_y_open * 100) if negative else 100 + (current_y_open * 100)
    img1.ellipse(((15 + (55*3),current_y_pos - 5),(25 + (55*3),current_y_pos + 5)), color_for_trade[current_x['trade']] , 2)

    current_date = current_x['date']
    img.save(f'output\\{current_date}.png')
    if current_x['trade'] == 'L':
        print('#---------Long(' + str(current_x['target_long']) + ')')
        print(current_x['long'])
        print(current_x['short'])
    if current_x['trade'] == 'S':
        print('#---------Short(' + str(current_x['target_short'])+ ')')
        print(current_x['long'])
        print(current_x['short'])